<a href="https://colab.research.google.com/github/pbr142/handson-ml2/blob/master/exercises/VAE_AI_Academy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Variational Autoencoder for Image Generation

In [1]:
from tensorflow import keras

In [2]:
keras.__version__

'2.4.0'

In [3]:
from google.colab import drive

In [4]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive
